Examining Results
=================
* [Importing Results](#importing-results)
* [Plot by date](#plot-by-date)


## Importing the results

In [ ]:
# Library imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime

In [ ]:
# Import the results
results = pd.read_csv('results_test.csv')
results = results.rename(columns={
    "Precipitation (kg/m^2/s)": "Precip",
    "Temperature (K)": "Temp",
    "Predictions": "Pred"
})

# Convert the date from ordinal to datetime
results['Date'] = results['Date'].apply(lambda x: datetime.datetime.fromordinal(x))

# Set index to date
results = results.set_index('Date')

results.head()

In [ ]:
# Extract the station locations from the original data
isoData = pd.read_csv("Isoscape_Data.csv")
stations = isoData[['Station', 'Lat', 'Long']].drop_duplicates()
stationList = stations['Station'].tolist()
stations.set_index('Station', inplace=True)

In [ ]:
# Separating the results by station into a dictionary
results_station = {}
for stat in stationList:
    results_station[stat] = results[results['Lat'] == stations.loc[stat, 'Lat']]

## Plot by date

In [ ]:
figure, axes = plt.subplots(5, 3, figsize=(20, 20))

for stat in stationList:
    data = results_station[stat]
    ax = axes.flat[stationList.index(stat)]
    ax.plot(data.index, data['Pred'], label='Predicted')
    ax.plot(data.index, data['Actual'], label='Actual')
    ax.xaxis.set_tick_params(rotation=45)
    ax.set_title(stat)


plt.subplots_adjust(wspace=0.2, hspace=0.5)

# Create a big subplot covering all the smaller ones and set its x-axis label
big_ax = figure.add_subplot(111, frameon=False)
big_ax.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
big_ax.set_xlabel('Date', labelpad=30, fontsize=20) 
big_ax.set_ylabel(r'$\delta$O18 (‰)', labelpad=30, fontsize=20)  

# Add a single legend for all subplots
handles, labels = ax.get_legend_handles_labels()
big_ax.legend(handles, labels, loc='upper right')

plt.show()
plt.close()

## Plotting Predictions vs. Actuals

In [ ]:
# Calculating the R^2 value
from sklearn.metrics import r2_score
coeff = r2_score(results['Actual'], results['Pred'])

# Plotting the results
plt.plot(results['Actual'], results['Pred'], 'o')
plt.plot(results['Actual'], results['Actual'], color='red')

# Placing the R^2 value in the plot
plt.text(0.1, 0.9, 'R^2 = ' + str(round(coeff, 2)),\
        ha='center',\
        va='center',\
        transform=plt.gca().transAxes,\
        bbox=dict(facecolor='red', alpha=0.3, edgecolor='black', linewidth=2))


# Labeling the axes
plt.title('Predicted vs Actual')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.show()
plt.close()

## Plot Precipitation and isotopes

In [ ]:
df = results_station["GOB"]
# Plotting the precipitation on one axis and the isotopes on the other
fig, ax1 = plt.subplots()

# Plotting the isotopes
ax1.set_xlabel('Date')
ax1.set_ylabel(r'$\delta$O18 (‰)')
ax1.plot(df.index, df['Actual'], label='Actual', color='Black')
ax1.plot(df.index, df['Pred'], label='Predicted', color='orange')
ax1.tick_params(axis='y')


# Plotting the precipitation
ax2 = ax1.twinx()
ax2.set_ylabel('Precipitation (kg/m^2/s)')
ax2.plot(df.index, df['Precip'], label='Precipitation', color='blue')
ax2.tick_params(axis='y')

ax1.xaxis.set_tick_params(rotation=45)

# Adding the legend
handles, labels = ax1.get_legend_handles_labels()
handles2, labels2 = ax2.get_legend_handles_labels()
handles.extend(handles2)
labels.extend(labels2)
ax1.legend(handles, labels, loc='upper right')
plt.show()

In [ ]:
df = results_station["GOB"]
# Plotting the precipitation on one axis and the isotopes on the other
fig, ax1 = plt.subplots()

# Plotting the isotopes
ax1.set_xlabel('Date')
ax1.set_ylabel(r'$\delta$O18 (‰)')
ax1.plot(df.index, df['Actual'], label='Actual', color='green')
ax1.plot(df.index, df['Pred'], label='Predicted', color='orange')
ax1.tick_params(axis='y')


# Plotting the precipitation
ax2 = ax1.twinx()
ax2.set_ylabel('Temperature (K)')
ax2.plot(df.index, df['Temp'], label='Temp', color='blue')
ax2.tick_params(axis='y')

ax1.xaxis.set_tick_params(rotation=45)

# Adding the legend
handles, labels = ax1.get_legend_handles_labels()
handles2, labels2 = ax2.get_legend_handles_labels()
handles.extend(handles2)
labels.extend(labels2)
ax1.legend(handles, labels, loc='upper right')

plt.show()